In [26]:
import torch
from utils import *       #changed
from models import TransformerLM, TransformerConditionedLM
import pickle
from imageio import imread
from PIL import Image
import torch.nn.functional as F

In [27]:
import yaml
# /net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/bleu-4_BEST_checkpoint_coco_2_cap_per_img_1_min_word_freq_gpu.pth.tar
# checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL/Trial_3/"
# checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/"
checkpoint_path = "../../saved_model/I2U/VC_5_captions/Trial_1/"

with open(checkpoint_path + 'config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

checkpoint = checkpoint_path + f'bleu-4_BEST_checkpoint_coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq_gpu.pth.tar'
dir_name = config["i2u"]["dir_name"]
model_params = config["i2u"]["model_params"]
train_params = config["i2u"]["train_params"]
data_folder = f'../../data/processed/{dir_name}/'  # folder with data files saved by create_input_files.py
# data_name = 'coco_4_cap_per_img_5_min_word_freq'  # base name shared by data files
#data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
# device = torch.device("cpu")

In [29]:
checkpoint = torch.load(checkpoint, map_location=device)

In [30]:
#checkpoint["model_state_dict"]["LM_decoder.layers.0.self_attn.in_proj_weight"].shape

torch.Size([3072, 1024])

In [32]:
for k,v in checkpoint["model_state_dict"].items():
    print(k)

embed.weight
pos_encoder.pe
LM_decoder.layers.0.self_attn.in_proj_weight
LM_decoder.layers.0.self_attn.in_proj_bias
LM_decoder.layers.0.self_attn.out_proj.weight
LM_decoder.layers.0.self_attn.out_proj.bias
LM_decoder.layers.0.linear1.weight
LM_decoder.layers.0.linear1.bias
LM_decoder.layers.0.linear2.weight
LM_decoder.layers.0.linear2.bias
LM_decoder.layers.0.norm1.weight
LM_decoder.layers.0.norm1.bias
LM_decoder.layers.0.norm2.weight
LM_decoder.layers.0.norm2.bias
LM_decoder.layers.1.self_attn.in_proj_weight
LM_decoder.layers.1.self_attn.in_proj_bias
LM_decoder.layers.1.self_attn.out_proj.weight
LM_decoder.layers.1.self_attn.out_proj.bias
LM_decoder.layers.1.linear1.weight
LM_decoder.layers.1.linear1.bias
LM_decoder.layers.1.linear2.weight
LM_decoder.layers.1.linear2.bias
LM_decoder.layers.1.norm1.weight
LM_decoder.layers.1.norm1.bias
LM_decoder.layers.1.norm2.weight
LM_decoder.layers.1.norm2.bias
LM_decoder.layers.2.self_attn.in_proj_weight
LM_decoder.layers.2.self_attn.in_proj_bias


In [5]:
for key, value in model.LM_decoder.state_dict().items():
    #for p in layer.modules():
    print(key, value.shape)
    #break

layers.0.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.self_attn.in_proj_bias torch.Size([3072])
layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.self_attn.out_proj.bias torch.Size([1024])
layers.0.linear1.weight torch.Size([4096, 1024])
layers.0.linear1.bias torch.Size([4096])
layers.0.linear2.weight torch.Size([1024, 4096])
layers.0.linear2.bias torch.Size([1024])
layers.0.norm1.weight torch.Size([1024])
layers.0.norm1.bias torch.Size([1024])
layers.0.norm2.weight torch.Size([1024])
layers.0.norm2.bias torch.Size([1024])
layers.1.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.1.self_attn.in_proj_bias torch.Size([3072])
layers.1.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.1.self_attn.out_proj.bias torch.Size([1024])
layers.1.linear1.weight torch.Size([4096, 1024])
layers.1.linear1.bias torch.Size([4096])
layers.1.linear2.weight torch.Size([1024, 4096])
layers.1.linear2.bias torch.Size([1024])
layers.1.norm1.weight torch.Size([10

In [6]:
for key, value in model.decoder.state_dict().items():
    #for p in layer.modules():
    print(key, value.shape)
    #break

layers.0.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.self_attn.in_proj_bias torch.Size([3072])
layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.self_attn.out_proj.bias torch.Size([1024])
layers.0.multihead_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.multihead_attn.in_proj_bias torch.Size([3072])
layers.0.multihead_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.multihead_attn.out_proj.bias torch.Size([1024])
layers.0.linear1.weight torch.Size([4096, 1024])
layers.0.linear1.bias torch.Size([4096])
layers.0.linear2.weight torch.Size([1024, 4096])
layers.0.linear2.bias torch.Size([1024])
layers.0.norm1.weight torch.Size([1024])
layers.0.norm1.bias torch.Size([1024])
layers.0.norm2.weight torch.Size([1024])
layers.0.norm2.bias torch.Size([1024])
layers.0.norm3.weight torch.Size([1024])
layers.0.norm3.bias torch.Size([1024])
layers.1.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.1.self_attn.in_proj_bias torch.Size([3072])
layers

### 注意：
encoder里的`norm2` 实际上是decoder里的`norm3`

encoder里的`dropout2` 实际上是decoder里的`dropout3`

所以直接 用 `load_state_dict(dict, strict = False)` 是不行的


In [25]:
decoder_state_dict = model.decoder.state_dict()
model.LM_decoder.load_state_dict(decoder_state_dict, strict=False)

AttributeError: 'NoneType' object has no attribute 'load_state_dict'

In [8]:
LM_decoder_state_dict = model.LM_decoder.state_dict()
model.decoder.load_state_dict(LM_decoder_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['layers.0.multihead_attn.in_proj_weight', 'layers.0.multihead_attn.in_proj_bias', 'layers.0.multihead_attn.out_proj.weight', 'layers.0.multihead_attn.out_proj.bias', 'layers.0.norm3.weight', 'layers.0.norm3.bias', 'layers.1.multihead_attn.in_proj_weight', 'layers.1.multihead_attn.in_proj_bias', 'layers.1.multihead_attn.out_proj.weight', 'layers.1.multihead_attn.out_proj.bias', 'layers.1.norm3.weight', 'layers.1.norm3.bias', 'layers.2.multihead_attn.in_proj_weight', 'layers.2.multihead_attn.in_proj_bias', 'layers.2.multihead_attn.out_proj.weight', 'layers.2.multihead_attn.out_proj.bias', 'layers.2.norm3.weight', 'layers.2.norm3.bias', 'layers.3.multihead_attn.in_proj_weight', 'layers.3.multihead_attn.in_proj_bias', 'layers.3.multihead_attn.out_proj.weight', 'layers.3.multihead_attn.out_proj.bias', 'layers.3.norm3.weight', 'layers.3.norm3.bias', 'layers.4.multihead_attn.in_proj_weight', 'layers.4.multihead_attn.in_proj_bias', 'layers.4.multihead_attn.out_p

In [14]:
from fairseq import options

In [16]:
parser = options.get_interactive_generation_parser()